In [64]:
def translate(seq, frame=1): 
    # frame: 1 = start at pos 0; 2 = start at pos 1; 3 = start at pos 2
    table = { 
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T', 
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                  
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P', 
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R', 
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A', 
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G', 
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L', 
        'TAC':'Y', 'TAT':'Y', 'TAA':'', 'TAG':'', 
        'TGC':'C', 'TGT':'C', 'TGA':'', 'TGG':'W', 
    } 
    protein = ''
    
    if frame == 1:
        start_val = 0
    if frame == 2:
        start_val = 1
    if frame == 3:
        start_val = 2
    
    for i in range(start_val, len(seq), 3): 
        try:
            codon = seq[i:i + 3] 
            protein+= table[codon] 
        except:
            break
    return protein

def split_translation (full_seq, len_cutoff = 10):
    sub_seq = [x for x in full_seq.split('_') if len(x) > len_cutoff]
    return sub_seq

def print_fasta (sub_list, prefix='test'):
    for idx, entry in enumerate(sub_list):
        print(f'>{prefix}_{idx}\n{entry}')
        
def save_fasta (sub_list, prefix, output):
    with open(output, 'w') as f:
        for idx, entry in enumerate(sub_list):
            line = f'>{prefix}_{idx}\n{entry}\n'
            f.write(line)


In [67]:
test_str = 'CCGAGATCTACACAACGCTATTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGTGAATTCTGGTGGGGCTTTTCGCAACGGGTTTGCCGCCAGAACACAGGTGTCGTGAAAACTACCCCTAAAAGCCAAAATGGAAAAAGGAAAAGACTCATATCAACATTGTCGTCATTGGACACGTAGATTCGGGCAAGTCCACCACTACTGGCCATCTGATCTATAAATGCGGTGGCATCGACAAAGAACCATTGAAAAATTTGAGAAGGAGGCTGCTGAGATGGGAAAGGGCTCCTTCAAGTATGCCTGGGTCTTGGATAAACTGAAAGCTGAGCGTGAACGTGGTATCACCATTGATATCTCCTTGTGGAAATTTGAGACCAGCAAGTACTATGTGACTATCATTGATGCCCCAGGACACAGAGACTTTATCAAAAACATGATTACAGGGACATCTCAGGCTGACTGTGCTGTCCTGATTGTTGCTGCTGGTGTTGGTGAATTTGAAGCTGGTATCTCCAGAATGGGCAGACCCGAGAGCATGCCCTTCTGAAGCACACTGGGTGTGAAACAACTAATTGTCGGTGTTAACAAAATGGATTCCACTGAGCCACCCTACAGCCAGAAGAGATATGAGGAAATTGTTAAGGAAGTCAGCACTTACATTAAGAAAATTGGCTACAACCCCGACACAGTAGCATTTGTGCCAATTTCTGGTTGGAATGGTGACAACATGCTGGAGCCAAGTGCTAACATGCCTTGGTTCAAGGGATGGAAAGTCACCCGTAAGGATGGCAATGCCAGTGGAACCACGCTGCTTGAGGCTCTGGACTGCATCCTACCACCAACTCGTCCAACTGACAAGCCCTTGCGCCTGCCTCTCCAGGATGTCTACAAAATTGGTGGTATTGGTACTGTTCCTGTTGGCCGAGTGGAGACTGGTGTTCTCAAACCCGGTATGGTGGTCACCTTTGCTCCAGTCAACGTTACAACGGAAGTAAAAATCTGTCGAAATGCACCATGAAGCTTTGAGTGAAAGCTCTTCCTGGGGACAATGTGGGCTTCAATGTCAAGAATGTGTCTGTCAGGATGTTCGTCGTGGCAACGTTGCTGGTGACAGCAAAAATGACCCACCAATGGAAGCAGCTGGCTTCACTGCTCAGGTGATTATCCTGAACCATCCAGGCCAAATAAGCGCCGGCTATGCCCCTGTATTGGATTGCCACACGGCTCACATTGCATGCAAGTTTGCTGAGCTGAAGGAAAAGATTGATCGCCGTTCTGGTAAAAGCTGGAAGATGGCCTAAATTCTTGAAGTCTGGTGATGCTGCCATTGTTGATATGGTTCCTGCAAGCCCATGTGTGTTGAGAGCTTCTCAGACTATCCACCTTTGGGTCGCTTTGCTGTTCGTGATATGAGACAGACAGTTGCGGTGGGTGTCATCAAAGCAGTGGACAAGAAGGCTGCTGGAGCTGGCAAGGTCACCAAGTCTGCCCAGAAAGCTCAGAAGGCTAAATGAATATTATCCCCTAATACCTGCCACCCCACTCTTAATCAGTGGTGGAAGAACGGTCTCAGAACTGTTTGTTTCAATTGGCCATTTAAGTTTAGTAGTAAAAAGACTGGTTAATGATAACAATGCATCGTAAAACCTTCAGAAGGAAAAGGAGAATGTTTTGTGGACCACTTTGGTTTTCTTTTTGCGTGTGGCAGTTTTAAGTTATTAGTTTTTAAAATCAGTACTTTTTAATGGAAACAACTTGACCAAAAATTTGTCACAGAATTTTGAGACCCATTAAAAAAGTTAAATGAGGAAAAAAAAAAAAAAAAAAAGTACTCTGCGTTGATACCACTGCTTAA'
prefix = 'test'
sub_list = []
for frame in [1,2,3]:
    full_seq = translate(test_str, frame)
    sub_list = sub_list+[full_seq]
save_fasta (sub_list, 'test', '/home/ubuntu/data/longread/translate/test_translation.fa')
    
    

In [70]:
# parse a fasta file and translate

## ingest with catch for line breaks
def ingest_fasta (filename):
    with open(filename, 'r') as f:
        for idx,line in enumerate(f):
            line_list = []
            if line.startswith('<'):
                entry_name = line
            else:
                line_list = line_list+[line.replace('\n','')]
            
                
            print(line)

In [71]:
ingest_fasta('/home/ubuntu/data/longread/A10_TSO1.fa')

>08d1f5b8-12fa-4995-990b-b9322805f849_12.14_2716_3_809_400

CTGAGATTCACACAACGCCATTCGTCGGCAGCGTCAGATGTATAAGAGACAGTGAATTCTGGTGGGGGCGACAAACCTACCGGAGCCTGGTGATAGCTGGCCGTCCAAGATAGAATCTTAGTTCAACTTTAAATTGCCCACAGAACCCCTTCTAAATCCCCTTGTAATTATCCGTTAGTCCAAAGAGGAACAGCTCTTGGACACTAGGAAAACCTTGTAGAGAGAGTAAAAATTTAACACCCAGGGCTCAAAAGCAGCCACCAATAAGAAAGCGTTCAAGCTCAACACCCACTACCTAAAAAATCCCAAACATATAACTGAACTCCTCACACCCACTGGACCAATCTATCACCCTATAGAAGAACTAATGTTAGTATAAGTAACATGAAAAACATTTCTCCTCCGCATAAGCCTGCGTCAGATCAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGTACTCTGCGCGATACCACTGCTTAAA

>6538a78d-3a56-4dcd-afa7-3c5f132071d5_11.62_4295_2_1516_1148

TGAGATCTTACACAACGCTATTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGTGAATTTTGGTGGGGACTAAACCAGCAATATGCAAACAAAGATGATAGCAAATGGCTCAGCTGAATTGGTAAAATGTCCGCTGTGTCTTTCTTTGTAAGACTGCAAATTGAGATCTGTTAATATCAACAATCCTTACAAAGAGTAAGAAATAATAACTGATTGCCTTTGTCTCTTCCCTTTTAATCTGACAGGGCCTTCAAGCACTTAGTAAGTGCATAGCAAATTCTCAAAGAACTAGCTAAATAATATTTTGGTCAATAAGAAATGATTTTGGCAAGTTTTATATCCAAAACTTATTTTTGGAATCACTTCGCATATTCCAATTTTATCTTCTTCATAAA

CCGAGATCTACACAACGCGATTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGTGAATTCTGGGGGGAGATGCACAGGAGGAGAAGCAGGAGCTGTCGGGAAGATCAGAAGCCAGTCATGGATGACCAGCGCGACCTATCTCCAACAATGAGCGGCTGACGGGCCGGGATCGCCCTGCCGCCCGCCCGCAAACCTTCAGCCCGGAGCCCTGTACACAGGCCCCGCCATCCGCCACCGCCCTCGGCCCGGCCACCACCGCCCCCTCCCAGCAGGGCCGGCTGGACAAACTGACATGTCCGAGTCCCAGAACCTGCGGCCCTGAGCATGAAGCTTCCCAAGCCTCCCAAGCCGTCAAGATGCGCATGGCCGCCCCTGCTGGTCTGATGCGCCCTGGCGCTGCCCATGGGACAGCCAAGAGCGGTACCGGCCCAGGGGCCATGTCAGAATGCGGCCGGAGCAGATCATGAAGTCCATCATCCCAGTGGTCCTGGCTGGCAGAATGCCATCTACGGCCTGAAGGTGTACCCGCCACTGAAGGGGAGCTTCCTGGAGAACCTGAGACACCTTAAGAACACCATGGAGACCATAGACTGGAAGGTCTTTGAGAGCTGGATGCACCATTGGCAACTGTTTGAAATGAGCAGGCATCAGCCTTGGAGCAAAAGCCCACTGACGCCCACCTGAGTCACTGGAACTGAGCGGCCTGTCAGCCGGGTTTGCCATCGGGATCTGGGGACCAGTCCCCATGTGAGGGCAGAGGCGCCCAACATCCCCAGCCCCATTCAGCTACATGATCCTTTGCTCCCTTCAGCCCCCAGCCCCGGCCATCTCCCATCCGCCCCTCATCCCATGAGACCCTGGTGCCGAGCCCACCAGCCACAGAATATTATGTAAAGACCAACCATTCCAGAACGAACAGCAGATAACAATACGCAAGGCCCGCTGCCCCAGTCAACAGGGGCTTGGTCCCAGGAAGTGTCCAAAAGTGCCCAGACAGAT

>f1dc0914-a5ed-4e1d-8fc2-c0e39f8ba3be_12.34_3599_3_767_382

CACCGAGATCTACAACGCTATTCGTCGGCAGCGTCAGATGTGTATAAGAGACAGTGAATTCTGGTGGGGCATTCAACATCAGGGGAGAGTCCTGTGGCCTGTTCTCTTAGTTTTCGTCTTCCTTATTTTGAGGCCAACAACCGTCATCATTCACTGTACCTTTGGAGCAAAAGAAGTTGCTGATCTAAACACCCTGAAAGGCAATGTTGTTATATATTTGGTAAAAGATTATAAACAGACTAGATGGAAACTTATTAGTGGTTAAGTTTAAAAACAAAACAAAAACCAAAAAAAACAGTCCAACTGGGCGCGGTGGCTCACGCCTATAATCCCAGCGCTTTGGGAACCAAGGCAGGCGGACCACGAGGTCAGGAGATCGGGACCATCCTGGCTTCTTCCAGTGAAACCCGTCTCTACAAAAAAAAAAAAAAAAAAGTACTCTGCGTTGATACCACTGCTTAA

>c395803b-8551-47e9-9e56-ef83712f5a0f_12.24_11131_13_822_394

CCGAGATCTACACAACGCTATTCGTCGGCAGCGTCAGATGTATAAGAGACAGTGAATTCTGGTGGGGGCGACAAACCTACCGAGCCTGGTGATAGCTGGTTGTCCAAGATAGAATCTTAGTTCAACTTTAAATTTGCCCACAGAACCCTCTAAATCCCCTTTGTAATTTGACTGTTAGTCCAAAGAGGAACAGCTCTTTGGACACTAGGAAAAAACCTTGTAGAGAGAGTAAAAAATTTAACACCCATAGTAGGCCTAAAAGCAGCCACCAATAAGAAAGCGTTCAAGCTCAACACCCACTACCTAAAAAAATCCCAAACATATAACTGAACTCCTCACACCCAATTGGACCAATCTATCACCCTATAGAAGAACTAATGTTAGTATAAGTAACATGAAAACATTCTCCTCCGC